In [1]:
import sys
sys.path.insert(0, "./../")

In [12]:
import pandas as pd

In [2]:
from app.mongodb_engine import MongoDB
from app.vertex_ai import TextEmbedding

In [3]:
mongo_db = MongoDB()

In [4]:
query = "How much is the range of Inflation Protected ?"

In [5]:
file_key = "chatpdf/user_2cotzAlVmd4jt0TlizIjUV0tzTi/4bba125b-4861-45e7-b50a-fda1e53d0de0/investment_policy_guidelines.pdf"

In [6]:
limit = 5

In [7]:
embedding_model = TextEmbedding()

In [8]:
query_vector = embedding_model([query])

100%|██████████| 1/1 [00:01<00:00,  1.09s/it]


In [9]:
query_vector = query_vector[0]

In [10]:
vector_search_result = mongo_db.vector_search(query_vector=query_vector,
                                              limit=limit, file_key=file_key)

In [11]:
keyword_search_result = mongo_db.keyword_search(query=query,
                                                limit=limit, file_key=file_key)

In [14]:
df_keyword = pd.DataFrame(keyword_search_result)

In [15]:
df_vector = pd.DataFrame(vector_search_result)

In [16]:
df_vector

,text,page_number,word_size,chunk_id,chat_id,file_key,score
0,Aim to minimize the probability of a decline i...,"[4, 5]",101,19,4bba125b-4861-45e7-b50a-fda1e53d0de0,chatpdf/user_2cotzAlVmd4jt0TlizIjUV0tzTi/4bba1...,0.812526
1,If the funding plan changes and funds are depo...,"[4, 5]",93,18,4bba125b-4861-45e7-b50a-fda1e53d0de0,chatpdf/user_2cotzAlVmd4jt0TlizIjUV0tzTi/4bba1...,0.810322
2,The objectives for the Operational Pool are as...,[4],117,13,4bba125b-4861-45e7-b50a-fda1e53d0de0,chatpdf/user_2cotzAlVmd4jt0TlizIjUV0tzTi/4bba1...,0.810300
3,Aim to minimize the probability of a decline i...,[4],84,15,4bba125b-4861-45e7-b50a-fda1e53d0de0,chatpdf/user_2cotzAlVmd4jt0TlizIjUV0tzTi/4bba1...,0.805786
4,30% of the portfolio is assumed to have potent...,[4],116,14,4bba125b-4861-45e7-b50a-fda1e53d0de0,chatpdf/user_2cotzAlVmd4jt0TlizIjUV0tzTi/4bba1...,0.804315


In [ ]:
def combine_vector_keyword_search(vector_search_results, keyword_search_results, limit: int = 5):

    if len(vector_search_results) == 0 and len(keyword_search_results) == 0:
        return []

    df_keyword_search = pd.DataFrame(keyword_search_results)
    df_vector_search = pd.DataFrame(vector_search_results)

    if len(vector_search_results) == 0:
        return vector_search_results

    if len(keyword_search_results) == 0:
        return keyword_search_results

    both_hit_indices = set(df_vector_search[['chunk_id']].merge(
        df_keyword_search[['chunk_id']])['chunk_id'])

    rest_size = limit - len(both_hit_indices)

    df_vector_search = normalized_score(df_vector_search)
    df_keyword_search = normalized_score(df_keyword_search)

    df_hybrid_search = pd.concat([df_keyword_search, df_vector_search])

    df_hybrid_search_1 = df_keyword_search[df_keyword_search['textIdx'].isin(
        both_hit_indices)].copy()

    df_hybrid_search_1['both_hit'] = True
    df_hybrid_search_2 = df_hybrid_search[~df_hybrid_search['textIdx'].isin(
        both_hit_indices)].copy()

    df_hybrid_search_2 = df_hybrid_search_2.sort_values(
        by='normalized_score', ascending=False).head(rest_size)
    df_hybrid_search_2['both_hit'] = False

    df_hybrid_search = pd.concat([df_hybrid_search_1, df_hybrid_search_2])

    results = df_hybrid_search.to_dict(orient='records')

    return results

In [ ]:
def normalized_score(df):
    max_score = max(df['score'])
    min_score = min(df['score'])
    if (max_score-min_score) == 0:
        df['normalized_score'] = 1
    else:
        df['normalized_score'] = df['score'].apply(
            lambda x: (x-min_score)/(max_score-min_score))
    return df